In [1]:
# importing the necessary libraries for dashboard
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
import plotly.express as px
import dash
from dash import dcc, html
import io
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from PIL import Image
import plotly.graph_objects as go

In [2]:
df = pd.read_csv(r"C:\Users\user\Desktop\#\DA\practice\python\E - commerce\cleaned_data sets\merged_data.csv", low_memory=False)

In [3]:
# changing all the columns to apporiate data type to enhance the visualisation
df['order_id'] = df['order_id'].astype(str)  # Identifier columns
df['customer_id'] = df['customer_id'].astype(str)
df['order_status'] = df['order_status'].astype(str)  # Categorical/text data
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], errors='coerce')  # DateTime
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'], errors='coerce')  # DateTime
df['order_delivered_carrier_date'] = pd.to_datetime(df['order_delivered_carrier_date'], errors='coerce')  # DateTime
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'], errors='coerce')  # DateTime
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'], errors='coerce')  # DateTime
df['order_item_id'] = pd.to_numeric(df['order_item_id'], errors='coerce')  # Integer
df['product_id'] = df['product_id'].astype(str)  # Identifier
df['seller_id'] = df['seller_id'].astype(str)  # Identifier
df['shipping_limit_date'] = pd.to_datetime(df['shipping_limit_date'], errors='coerce')  # DateTime
df['price'] = pd.to_numeric(df['price'], errors='coerce')  # Numeric
df['freight_value'] = pd.to_numeric(df['freight_value'], errors='coerce')  # Numeric
df['product_category_name'] = df['product_category_name'].astype(str)  # Categorical/text data
df['product_weight_g'] = pd.to_numeric(df['product_weight_g'], errors='coerce')  # Numeric
df['customer_unique_id'] = df['customer_unique_id'].astype(str)  # Identifier
df['customer_zip_code_prefix'] = df['customer_zip_code_prefix'].astype(str)  # Categorical/text data
df['customer_city'] = df['customer_city'].astype(str)  # Categorical/text data
df['customer_state'] = df['customer_state'].astype(str)  # Categorical/text data
df['payment_sequential'] = pd.to_numeric(df['payment_sequential'], errors='coerce')  # Numeric
df['payment_type'] = df['payment_type'].astype(str)  # Categorical/text data
df['payment_installments'] = pd.to_numeric(df['payment_installments'], errors='coerce')  # Numeric
df['payment_value'] = pd.to_numeric(df['payment_value'], errors='coerce')  # Numeric
df['seller_zip_code_prefix'] = df['seller_zip_code_prefix'].astype(str)  # Categorical/text data
df['seller_city'] = df['seller_city'].astype(str)  # Categorical/text data
df['seller_state'] = df['seller_state'].astype(str)  # Categorical/text data
df['product_category_name_eng'] = df['product_category_name_eng'].astype(str)  # Categorical/text data


In [4]:
# Remove unwanted values from the payment_type column
df = df[~df['payment_type'].isin(['not_defined', 'UNKNOWN'])]

# Alternatively, you can use boolean indexing
# df = df[(df['payment_type'] != 'not_defined') & (df['payment_type'] != 'UNKNOWN')]


In [5]:
# Aggregating sales performance by day, month, quarter, and year
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Daily sales performance
sales_performance_daily = df.groupby(df['order_purchase_timestamp'].dt.date)['price'].sum().reset_index()
sales_performance_daily.columns = ['order_purchase_timestamp', 'total_sales']

# Monthly sales performance
sales_performance_monthly = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['price'].sum().reset_index()
sales_performance_monthly.columns = ['order_purchase_timestamp', 'total_sales']

# Quarterly sales performance
sales_performance_quarterly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Q'))['price'].sum().reset_index()
sales_performance_quarterly.columns = ['order_purchase_timestamp', 'total_sales']

# Yearly sales performance
sales_performance_yearly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Y'))['price'].sum().reset_index()
sales_performance_yearly.columns = ['order_purchase_timestamp', 'total_sales']


In [10]:
html.Div([
    html.H2("Payment Methods Analysis", style={'textAlign': 'left'}),
    dcc.Graph(
        id='payment-distribution',
        figure=update_layout(px.pie(payment_distribution, names='payment_type', values='count', title='Payment Method Distribution', hole=0.3))
    ),
], style={'padding': '20px'}),


(Div(children=[H2(children='Payment Methods Analysis', style={'textAlign': 'left'}), Graph(id='payment-distribution', figure=Figure({
     'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
               'hole': 0.3,
               'hovertemplate': 'payment_type=%{label}<br>count=%{value}<extra></extra>',
               'labels': array(['boleto', 'credit_card', 'debit_card', 'voucher'], dtype=object),
               'legendgroup': '',
               'name': '',
               'showlegend': True,
               'type': 'pie',
               'values': array([23037, 87286,  1698,  6407], dtype=int64)}],
     'layout': {'font': {'family': 'Arial', 'size': 12},
                'legend': {'tracegroupgap': 0},
                'paper_bgcolor': 'white',
                'plot_bgcolor': 'white',
                'template': '...',
                'title': {'text': 'Payment Method Distribution'},
                'xaxis': {'showgrid': False, 'zeroline': False},
                'yaxis': {'showg

In [6]:
# Aggregating the distribution of payment methods
payment_distribution = df.groupby('payment_type')['payment_type'].count().reset_index(name='count')


In [14]:

# For this example, let's assume df is already loaded and contains the necessary columns

# Convert date columns to datetime format
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'])

# ---------------- Data Preparation ----------------

# 1. Sales Performance Data
# Daily sales performance
sales_performance_daily = df.groupby(df['order_purchase_timestamp'].dt.date)['price'].sum().reset_index()
sales_performance_daily.columns = ['order_purchase_timestamp', 'total_sales']

# Monthly sales performance
sales_performance_monthly = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['price'].sum().reset_index()
sales_performance_monthly.columns = ['order_purchase_timestamp', 'total_sales']

# Quarterly sales performance
sales_performance_quarterly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Q'))['price'].sum().reset_index()
sales_performance_quarterly.columns = ['order_purchase_timestamp', 'total_sales']

# Yearly sales performance
sales_performance_yearly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Y'))['price'].sum().reset_index()
sales_performance_yearly.columns = ['order_purchase_timestamp', 'total_sales']

# 2. Payment Distribution
payment_distribution = df.groupby('payment_type').size().reset_index(name='count')

# 3. State Revenue and Top 10 States
state_revenue = df.groupby('customer_state')['price'].sum().reset_index(name='total_revenue')
top_10_states = state_revenue.sort_values(by='total_revenue', ascending=False).head(10)

# 4. Sales by Day of the Week
df['order_day_of_week'] = df['order_purchase_timestamp'].dt.day_name()
sales_by_day = df.groupby('order_day_of_week')['price'].sum().reset_index(name='total_sales')

# 5. Correlation Data for Price and Freight Value
correlation_data = df[['price', 'freight_value']].dropna()

# 6. Customer Acquisition Over Time
customer_acquisition = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['customer_unique_id'].nunique().reset_index()
customer_acquisition.columns = ['order_purchase_timestamp', 'customer_count']

# 7. Order Distribution by Hour
df['order_hour'] = df['order_purchase_timestamp'].dt.hour
order_distribution_hour = df.groupby('order_hour').size().reset_index(name='total_orders')

# 8. Average Delivery Time and Delayed Orders Percentage
df['delivery_time'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
average_delivery_time = df['delivery_time'].mean()

delayed_orders = df[df['order_delivered_customer_date'] > df['order_estimated_delivery_date']]
delayed_orders_percentage = (len(delayed_orders) / len(df)) * 100

# 9. Top 10 Cities by Order Volume
top_10_cities = df.groupby('customer_city').size().reset_index(name='order_volume').sort_values(by='order_volume', ascending=False).head(10)

# 10. Average Order Value (AOV)
aov = df['price'].mean()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define a function to update layout for all Plotly charts
def update_layout(fig):
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial", size=12),
    )
    return fig

# Convert Matplotlib figure to Plotly figure
def matplotlib_to_plotly(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)
    img_data = np.array(img)
    plotly_fig = go.Figure(go.Image(z=img_data))
    plt.close(fig)
    return plotly_fig

# 1. Treemap for Top 10 Product Categories by Sales
top_10_categories = df.groupby('product_category_name_eng')['price'].sum().nlargest(10).reset_index()

treemap_fig = px.treemap(top_10_categories, 
                         path=['product_category_name_eng'],  
                         values='price',                      
                         color='price',                      
                         color_continuous_scale='Blues',     
                         title='Top 10 Product Categories by Sales')

treemap_fig.update_traces(
    textinfo='label+value',  
    texttemplate='%{label}<br>$%{value:,.2f}' 
)

treemap_fig.update_layout(
    title_font_size=15,
    title_x=0.5,                            
    font=dict(size=12),                     
    coloraxis_colorbar=dict(title="Total Sales"),  
    margin=dict(t=50, l=25, r=25, b=25)    
)

# 2. Scatter Plot for Correlation Between Price and Freight Value (Matplotlib)
def correlation_price_freight_plot():
    fig, ax = plt.subplots(figsize=(10,6))
    ax.scatter(df['price'], df['freight_value'], alpha=0.6, c='dodgerblue', edgecolor='k', linewidth=0.5)
    ax.set_title('Correlation Between Price and Freight Value', fontsize=14, fontweight='bold', color='darkblue')
    ax.set_xlabel('Price (in currency)', fontsize=12, fontweight='semibold')
    ax.set_ylabel('Freight Value (in currency)', fontsize=12, fontweight='semibold')
    ax.grid(False)
    sns.despine()
    return matplotlib_to_plotly(fig)

# 3. Scatter Plot for Number of Orders vs Average Payment Value (Matplotlib)
def orders_vs_avg_payment_plot():
    customer_orders = df.groupby('customer_id')['order_id'].count()
    customer_payment = df.groupby('customer_id')['payment_value'].mean()

    fig, ax = plt.subplots(figsize=(10,6))
    ax.scatter(customer_orders, customer_payment, color='#117554')
    ax.set_title('Number of Orders vs Average Payment Value', fontsize=9, fontweight=550)
    ax.set_xlabel("Number of Orders '000'", fontsize=9, fontweight=500)
    ax.set_ylabel('Average Payment Value', fontsize=9, fontweight=500)
    sns.despine()
    return matplotlib_to_plotly(fig)

# 4. Bar Plot for Top 10 Product Categories by Average Delivery Time (Matplotlib)
def top_10_delivery_time_plot():
    avg_delivery_time = df.groupby('product_category_name_eng')['delivery_time'].mean()
    top_10_products = avg_delivery_time.sort_values(ascending=False).head(10)

    fig, ax = plt.subplots(figsize=(10,6))
    top_10_products.plot(kind='barh', color='#117554', ax=ax)
    for p in ax.patches:
        ax.annotate(f'{p.get_width():.1f}', (p.get_width(), p.get_y() + p.get_height()/2.),
                    ha='left', va='center', fontsize=11, color='black', fontweight='semibold')
    ax.set_title('Top 10 Product Categories by Average Delivery Time', fontsize=14, fontweight='semibold')
    ax.set_xlabel('Average Delivery Time (days)', fontsize=12, fontweight='semibold')
    ax.set_ylabel('Product Category', fontsize=12, fontweight='semibold')
    ax.xaxis.set_visible(False)
    ax.grid(False)
    sns.despine(left=True)
    return matplotlib_to_plotly(fig)

# Define the layout of the dashboard
app.layout = html.Div(style={'fontFamily': 'Arial'}, children=[
    html.Div([
        html.H1("Baana Group Sales Dashboard", style={'textAlign': 'center', 'color': '#000', 'backgroundColor': '#fff', 'padding': '10px'}),
    ]),
    
    html.Div([
        html.H2("Sales Performance Analysis", style={'textAlign': 'left'}),
        dcc.Graph(
            id='daily-sales',
            figure=update_layout(px.line(sales_performance_daily, x='order_purchase_timestamp', y='total_sales', title='Total Sales by Day', markers=True))
        ),
        dcc.Graph(
            id='monthly-sales',
            figure=update_layout(px.line(sales_performance_monthly, x='order_purchase_timestamp', y='total_sales', title='Total Sales by Month', markers=True))
        ),
        dcc.Graph(
            id='quarterly-sales',
            figure=update_layout(px.line(sales_performance_quarterly.sort_values('order_purchase_timestamp'), 
                           x='order_purchase_timestamp', 
                           y='total_sales', 
                           title='Total Sales by Quarter',
                           markers=True))
        ),
        dcc.Graph(
            id='yearly-sales',
            figure=update_layout(px.line(sales_performance_yearly.sort_values('order_purchase_timestamp'), 
                           x='order_purchase_timestamp', 
                           y='total_sales', 
                           title='Total Sales by Year',
                           markers=True))
        ),
        dcc.Graph(
            id='top-10-categories',
            figure=treemap_fig  
        ),
        html.P(f"Average Order Value (AOV): ${aov:.2f}", style={'fontSize': '20px'}),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Payment Methods Analysis", style={'textAlign': 'left'}),
        dcc.Graph(
            id='payment-distribution',
            figure=update_layout(px.pie(payment_distribution, names='payment_type', values='count', title='Payment Method Distribution', hole=0.3))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Order Fulfillment and Delivery Efficiency", style={'textAlign': 'left'}),
        html.P(f"Average Delivery Time: {average_delivery_time:.2f} days", style={'fontSize': '20px'}),
        html.P(f"Percentage of Delayed Orders: {delayed_orders_percentage:.2f}%", style={'fontSize': '20px'}),
        dcc.Graph(
            id='top-10-delivery-time',
            figure=top_10_delivery_time_plot()
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Geographic Insights", style={'textAlign': 'left'}),
        dcc.Graph(
            id='sales-by-states',
            figure=update_layout(px.treemap(state_revenue, path=['customer_state'], values='total_revenue', title='Sales by State'))
        ),
        dcc.Graph(
            id='top-10-states',
            figure=update_layout(px.bar(top_10_states, x='customer_state', y='total_revenue', title='Top 10 States by Revenue', color='total_revenue'))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Sales by Day of the Week", style={'textAlign': 'left'}),
        dcc.Graph(
            id='sales-by-day',
            figure=update_layout(px.bar(sales_by_day, x='order_day_of_week', y='total_sales', title='Sales by Day of the Week', color='total_sales'))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Advanced Metrics", style={'textAlign': 'left'}),
        dcc.Graph(
            id='correlation-price-freight',
            figure=correlation_price_freight_plot()
        ),
        dcc.Graph(
            id='orders-vs-avg-payment',
            figure=orders_vs_avg_payment_plot()
        ),
        dcc.Graph(
            id='heatmap-order-distribution',
            figure=update_layout(px.density_heatmap(order_distribution_hour, x='order_hour', y='total_orders', title='Order Distribution by Hour of Day'))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Customer Insights", style={'textAlign': 'left'}),
        dcc.Graph(
            id='customer-acquisition',
            figure=update_layout(px.line(customer_acquisition, x='order_purchase_timestamp', y='customer_count', title='Customer Acquisition Over Time', markers=True))
        ),
        dcc.Graph(
            id='top-10-cities',
            figure=update_layout(px.bar(top_10_cities, x='customer_city', y='order_volume', title='Top 10 Cities by Order Volume', color='order_volume'))
        ),
    ], style={'padding': '20px'}),
])

if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
TypeError: Object of type Period is not JSON serializable



In [7]:
# Monthly sales performance - convert Period to Timestamp
sales_performance_monthly = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['price'].sum().reset_index()
sales_performance_monthly['order_purchase_timestamp'] = sales_performance_monthly['order_purchase_timestamp'].dt.to_timestamp()
sales_performance_monthly.columns = ['order_purchase_timestamp', 'total_sales']

# Quarterly sales performance - convert Period to Timestamp
sales_performance_quarterly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Q'))['price'].sum().reset_index()
sales_performance_quarterly['order_purchase_timestamp'] = sales_performance_quarterly['order_purchase_timestamp'].dt.to_timestamp()
sales_performance_quarterly.columns = ['order_purchase_timestamp', 'total_sales']

# Yearly sales performance - convert Period to Timestamp
sales_performance_yearly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Y'))['price'].sum().reset_index()
sales_performance_yearly['order_purchase_timestamp'] = sales_performance_yearly['order_purchase_timestamp'].dt.to_timestamp()
sales_performance_yearly.columns = ['order_purchase_timestamp', 'total_sales']


In [8]:
# Monthly sales performance - convert Period to string format
sales_performance_monthly = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['price'].sum().reset_index()
sales_performance_monthly['order_purchase_timestamp'] = sales_performance_monthly['order_purchase_timestamp'].astype(str)
sales_performance_monthly.columns = ['order_purchase_timestamp', 'total_sales']

# Quarterly sales performance - convert Period to string format
sales_performance_quarterly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Q'))['price'].sum().reset_index()
sales_performance_quarterly['order_purchase_timestamp'] = sales_performance_quarterly['order_purchase_timestamp'].astype(str)
sales_performance_quarterly.columns = ['order_purchase_timestamp', 'total_sales']

# Yearly sales performance - convert Period to string format
sales_performance_yearly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Y'))['price'].sum().reset_index()
sales_performance_yearly['order_purchase_timestamp'] = sales_performance_yearly['order_purchase_timestamp'].astype(str)
sales_performance_yearly.columns = ['order_purchase_timestamp', 'total_sales']


# Dashboard

In [14]:

# Assuming df is the preprocessed DataFrame
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])

# Daily sales performance
sales_performance_daily = df.groupby(df['order_purchase_timestamp'].dt.date)['price'].sum().reset_index()
sales_performance_daily.columns = ['order_purchase_timestamp', 'total_sales']

# Monthly sales performance
sales_performance_monthly = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['price'].sum().reset_index()
sales_performance_monthly['order_purchase_timestamp'] = sales_performance_monthly['order_purchase_timestamp'].astype(str)
sales_performance_monthly.columns = ['order_purchase_timestamp', 'total_sales']

# Quarterly sales performance
sales_performance_quarterly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Q'))['price'].sum().reset_index()
sales_performance_quarterly['order_purchase_timestamp'] = sales_performance_quarterly['order_purchase_timestamp'].astype(str)
sales_performance_quarterly.columns = ['order_purchase_timestamp', 'total_sales']

# Yearly sales performance
sales_performance_yearly = df.groupby(df['order_purchase_timestamp'].dt.to_period('Y'))['price'].sum().reset_index()
sales_performance_yearly['order_purchase_timestamp'] = sales_performance_yearly['order_purchase_timestamp'].astype(str)
sales_performance_yearly.columns = ['order_purchase_timestamp', 'total_sales']

# Payment method distribution
payment_distribution = df.groupby('payment_type')['payment_type'].count().reset_index(name='count')

# Sales by product category
sales_by_category = df.groupby('product_category_name_eng')['price'].sum().reset_index().sort_values('price', ascending=False)

# Sales by customer state
sales_by_customer_state = df.groupby('customer_state')['price'].sum().reset_index().sort_values('price', ascending=False)

# Sales by seller state
sales_by_seller_state = df.groupby('seller_state')['price'].sum().reset_index().sort_values('price', ascending=False)

# Order status distribution
order_status_distribution = df.groupby('order_status')['order_status'].count().reset_index(name='count')

# Average delivery time by month
df['delivery_time'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days
avg_delivery_time = df.groupby(df['order_purchase_timestamp'].dt.to_period('M'))['delivery_time'].mean().reset_index()
avg_delivery_time['order_purchase_timestamp'] = avg_delivery_time['order_purchase_timestamp'].astype(str)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define a function to update layout for all Plotly charts
def update_layout(fig):
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(family="Arial", size=12),
    )
    return fig

# Define the layout of the dashboard
app.layout = html.Div(style={'fontFamily': 'Arial'}, children=[
    html.Div([
        html.H1("Baana Group Sales Dashboard", style={'textAlign': 'center', 'color': '#000', 'backgroundColor': '#fff', 'padding': '10px'}),
    ]),

    html.Div([
        html.H2("Sales Performance Analysis", style={'textAlign': 'left' , 'color': '#000', 'backgroundColor': '#fff', 'padding': '10px'}),
        dcc.Graph(
            id='daily-sales',
            figure=update_layout(px.line(sales_performance_daily, x='order_purchase_timestamp', y='total_sales', title='Total Sales by Day', markers=True))
        ),
        dcc.Graph(
            id='monthly-sales',
            figure=update_layout(px.line(sales_performance_monthly, x='order_purchase_timestamp', y='total_sales', title='Total Sales by Month', markers=True))
        ),
        dcc.Graph(
            id='quarterly-sales',
            figure=update_layout(px.line(sales_performance_quarterly.sort_values('order_purchase_timestamp'), 
                           x='order_purchase_timestamp', 
                           y='total_sales', 
                           title='Total Sales by Quarter',
                           markers=True))
        ),
        dcc.Graph(
            id='yearly-sales',
            figure=update_layout(px.line(sales_performance_yearly.sort_values('order_purchase_timestamp'), 
                           x='order_purchase_timestamp', 
                           y='total_sales', 
                           title='Total Sales by Year',
                           markers=True))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Additional Insights", style={'textAlign': 'left'}),
        dcc.Graph(
            id='payment-distribution',
            figure=update_layout(px.pie(payment_distribution, names='payment_type', values='count', title='Payment Method Distribution', hole=0.3))
        ),
        dcc.Graph(
            id='sales-by-category',
            figure=update_layout(px.bar(sales_by_category, x='product_category_name_eng', y='price', title='Total Sales by Product Category'))
        ),
        dcc.Graph(
            id='sales-by-customer-state',
            figure=update_layout(px.bar(sales_by_customer_state, x='customer_state', y='price', title='Total Sales by Customer State'))
        ),
        dcc.Graph(
            id='sales-by-seller-state',
            figure=update_layout(px.bar(sales_by_seller_state, x='seller_state', y='price', title='Total Sales by Seller State'))
        ),
        dcc.Graph(
            id='order-status-distribution',
            figure=update_layout(px.pie(order_status_distribution, names='order_status', values='count', title='Order Status Distribution', hole=0.3))
        ),
        dcc.Graph(
            id='avg-delivery-time',
            figure=update_layout(px.line(avg_delivery_time, x='order_purchase_timestamp', y='delivery_time', title='Average Delivery Time by Month', markers=True))
        ),
    ], style={'padding': '20px'}),
])

if __name__ == '__main__':
    app.run_server(debug=True)

# Dashboard 2

In [11]:

# Sample Data Preparation (Assuming your data processing is done)
# df = pd.read_csv('your_data.csv')  # Load your data here
# Perform any necessary data preprocessing...

# Initialize the Dash app
app = dash.Dash(__name__)

# Update layout for a professional appearance
def update_layout(fig):
    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig.update_layout(
        plot_bgcolor='#f9f9f9',  # Light gray background
        paper_bgcolor='white',    # White for the paper background
        font=dict(family="Helvetica, Arial", size=12),
        title_font=dict(size=14, color='#333'),  # Darker title font
        coloraxis_colorbar=dict(title="Total Sales", title_font=dict(size=12)),
    )
    return fig

# Define a function to create treemap for top 10 categories
def create_treemap():
    top_10_categories = df.groupby('product_category_name_eng')['price'].sum().nlargest(10).reset_index()
    treemap_fig = px.treemap(top_10_categories, 
                             path=['product_category_name_eng'],  
                             values='price',                     
                             color='price',                      
                             color_continuous_scale='Blues',    
                             title='Top 10 Product Categories by Sales')

    treemap_fig.update_traces(
        textinfo='label+value',  
        texttemplate='%{label}<br>$%{value:,.2f}' 
    )

    return update_layout(treemap_fig)

# Define the layout of the dashboard
app.layout = html.Div(style={'fontFamily': 'Helvetica, Arial', 'backgroundColor': '#f9f9f9'}, children=[
    html.Div([
        html.H1("Baana Group Sales Dashboard", style={'textAlign': 'center', 'color': '#333', 'padding': '20px'}),
    ]),

    html.Div([
        html.H2("Sales Performance Analysis", style={'textAlign': 'left'}),
        dcc.Graph(
            id='top-10-categories',
            figure=create_treemap()  # Updated treemap figure here
        ),
    ], style={'padding': '20px'}),

    # Add more sections here, following a similar pattern
    html.Div([
        html.H2("Payment Methods Analysis", style={'textAlign': 'left'}),
        dcc.Graph(
            id='payment-distribution',
            figure=update_layout(px.pie(payment_distribution, names='payment_type', values='count', title='Payment Method Distribution', hole=0.3))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Order Fulfillment and Delivery Efficiency", style={'textAlign': 'left'}),
        html.P(f"Average Delivery Time: {average_delivery_time:.2f} days", style={'fontSize': '20px'}),
        html.P(f"Percentage of Delayed Orders: {delayed_orders_percentage:.2f}%", style={'fontSize': '20px'}),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Geographic Insights", style={'textAlign': 'left'}),
        dcc.Graph(
            id='sales-by-states',
            figure=update_layout(px.treemap(state_revenue, path=['customer_state'], values='total_revenue', title='Sales by State'))
        ),
        dcc.Graph(
            id='top-10-states',
            figure=update_layout(px.bar(top_10_states, x='customer_state', y='total_revenue', title='Top 10 States by Revenue', color='total_revenue'))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Sales by Day of the Week", style={'textAlign': 'left'}),
        dcc.Graph(
            id='sales-by-day',
            figure=update_layout(px.bar(sales_by_day.reset_index(), x='order_day_of_week', y='total_sales', title='Sales by Day of the Week', color='total_sales'))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Advanced Metrics", style={'textAlign': 'left'}),
        dcc.Graph(
            id='correlation-price-freight',
            figure=update_layout(px.scatter(correlation_data, x='price', y='freight_value', title='Correlation Between Price and Freight Value'))
        ),
        dcc.Graph(
            id='heatmap-order-distribution',
            figure=update_layout(px.density_heatmap(order_distribution_hour, x='order_hour', y='total_orders', title='Order Distribution by Hour of Day'))
        ),
    ], style={'padding': '20px'}),

    html.Div([
        html.H2("Customer Insights", style={'textAlign': 'left'}),
        dcc.Graph(
            id='customer-acquisition',
            figure=update_layout(px.line(customer_acquisition, x='order_purchase_timestamp', y='customer_count', title='Customer Acquisition Over Time', markers=True))
        ),
        dcc.Graph(
            id='top-10-cities',
            figure=update_layout(px.bar(top_10_cities, x='customer_city', y='order_volume', title='Top 10 Cities by Order Volume', color='order_volume'))
        ),
    ], style={'padding': '20px'}),
])

if __name__ == '__main__':
    app.run_server(debug=True)

NameError: name 'average_delivery_time' is not defined

In [20]:
# Sample Data Preparation (Assuming your data processing is done)
# df = pd.read_csv('your_data.csv')  # Load your data here

# Convert Period columns to strings if any
# df['order_purchase_timestamp'] = df['order_purchase_timestamp'].astype(str)

# Initialize the Dash app
app = dash.Dash(__name__)

# ... [rest of your dashboard code]

if __name__ == '__main__':
    app.run_server(debug=True)


---------------------------------------------------------------------------
NoLayoutException                         Traceback (most recent call last)
NoLayoutException: The layout was `None` at the time that `run_server` was called.
Make sure to set the `layout` attribute of your application
before running the server.



In [12]:
# Assuming df is your main DataFrame
df['order_purchase_timestamp'] = df['order_purchase_timestamp'].astype(str)
# If you have other period columns, convert them similarly
# df['some_period_column'] = df['some_period_column'].astype(str)


In [ ]:
# Sample Data Preparation (Assuming your data processing is done)
# df = pd.read_csv('your_data.csv')  # Load your data here

# Convert Period columns to strings if any
# df['order_purchase_timestamp'] = df['order_purchase_timestamp'].astype(str)

# Initialize the Dash app
app = dash.Dash(__name__)

# ... [rest of your dashboard code]

if __name__ == '__main__':
    app.run_server(debug=True)
